In [18]:
import matplotlib
matplotlib.use('TkAgg')
import ecg_noise_detector.src.ecg_noise_detector.noiseDetector as nd

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC

In [3]:
import importlib
importlib.reload(nd)


<module 'ecg_noise_detector.src.ecg_noise_detector.noiseDetector' from 'C:\\Users\\daniel\\Documents\\CambridgeSoftwareProjects\\ecg-signal-quality\\ecg_noise_detector\\src\\ecg_noise_detector\\noiseDetector.py'>

In [4]:
dataset = pd.read_pickle("CinC2017Data/database.pk")

In [5]:
dataset["length"] = dataset["data"].map(lambda arr: arr.shape[-1])
# select only the 30s length records
dataset = dataset[dataset["length"] == 9000]
dataset["data"] = dataset["data"].map(lambda d: d[0])

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

In [8]:
# dataset["onehot"] = dataset["class"].map(generate_onehot)
def generate_index(c):
    if c == "N":
        return 0
    if c == "O":
        return 0
    if c == "A":
        return 0
    if c == "~":
        return 1

dataset["class_index"] = dataset["class"].map(generate_index)

train_dataset, test_dataset = train_test_split(dataset, test_size=0.15, stratify=dataset["class_index"])

print(len(test_dataset.index))

897


In [8]:
print(dataset["class"].value_counts())

N    3695
O    1655
A     504
~     123
Name: class, dtype: int64


In [18]:
print(len(nd.get_example_ecg()))

15200


In [19]:
lt_yoav = nd._length_transfrom(test_dataset["data"][0], 100)

In [20]:
lt_mine = nd._length_transform_faster(test_dataset["data"][0], 100)

In [53]:
def test_code():
    ecgs = test_dataset["data"].head(20)
    nd.is_noisy_batch(ecgs, fs=300)

import cProfile
cProfile.run('test_code()')

C:\Users\daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.0.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\daniel\Documents\CambridgeSoftwareProjects\ecg-signal-quality\ecg_noise_detector\src\ecg_noise_detector\noiseDetector.py:81: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  p = pd.Series(index=ecgs.index)


Processing ecg: A04315
Processing ecg: A03779
Processing ecg: A05397
Processing ecg: A03823
Processing ecg: A00227
Processing ecg: A08170
Processing ecg: A03236
Processing ecg: A07489
Processing ecg: A04197
Processing ecg: A00860
Processing ecg: A04877
Processing ecg: A05200
Processing ecg: A01613
Processing ecg: A02253
Processing ecg: A05897
Processing ecg: A05041
Processing ecg: A07776
Processing ecg: A06002
Processing ecg: A07440
Processing ecg: A01792
         5192772 function calls (5113124 primitive calls) in 4.064 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.064    4.064 3875118959.py:1(test_code)
     2840    0.001    0.000    0.012    0.000 <__array_function__ internals>:177(all)
     3360    0.004    0.000    0.021    0.000 <__array_function__ internals>:177(amax)
     8480    0.015    0.000    0.058    0.000 <__array_function__ internals>:177(any)
     1720    0.001    0.000   

In [23]:
ecgs = test_dataset["data"]
test_dataset["predictions"] = nd.is_noisy_batch(ecgs, fs=300, filter=False)

C:\Users\daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.0.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\daniel\Documents\CambridgeSoftwareProjects\ecg-signal-quality\ecg_noise_detector\src\ecg_noise_detector\noiseDetector.py:81: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  p = pd.Series(index=ecgs.index)


Processing ecg: A04251
Processing ecg: A06632
Processing ecg: A06751
Processing ecg: A05406
Processing ecg: A00426
Processing ecg: A00549
Processing ecg: A00199
Processing ecg: A07050
Processing ecg: A06139
Processing ecg: A04279
Processing ecg: A02611
Processing ecg: A01097
Processing ecg: A03639
Processing ecg: A02656
Processing ecg: A06985
Processing ecg: A04226
Processing ecg: A00018
Processing ecg: A06079
Processing ecg: A04949
Processing ecg: A06459
Processing ecg: A03365
Processing ecg: A07771
Processing ecg: A00802
Processing ecg: A02654
Processing ecg: A07266
Processing ecg: A01383
Processing ecg: A04783
Processing ecg: A05554
Processing ecg: A06395
Processing ecg: A07808
Processing ecg: A06954
Processing ecg: A00740
Processing ecg: A04323
Processing ecg: A01475
Processing ecg: A00902
Processing ecg: A07685
Processing ecg: A03136
Processing ecg: A02560
Processing ecg: A08381
Processing ecg: A07451
Processing ecg: A07111
Processing ecg: A07551
Processing ecg: A07710
Processing 

In [30]:
print(test_dataset.head(50))

       class                                               data  length  \
A04251     ~  [584, 597, 577, 626, 659, 725, 772, 787, 797, ...    9000   
A06632     N  [-10, -12, -14, -17, -19, -21, -22, -23, -23, ...    9000   
A06751     N  [408, 484, 560, 633, 697, 729, 725, 694, 665, ...    9000   
A05406     N  [140, 174, 210, 251, 280, 293, 300, 306, 309, ...    9000   
A00426     N  [-20, -24, -31, -38, -44, -47, -47, -45, -41, ...    9000   
A00549     O  [-575, -695, -819, -941, -1046, -1079, -1091, ...    9000   
A00199     N  [24, 28, 40, 54, 62, 60, 55, 50, 45, 39, 33, 2...    9000   
A07050     N  [-24, -34, -67, -89, -111, -132, -147, -156, -...    9000   
A06139     N  [61, 65, 63, 51, 44, 39, 35, 32, 30, 29, 29, 2...    9000   
A04279     O  [-1072, -1282, -1492, -1692, -1857, -1942, -19...    9000   
A02611     A  [-23, -35, -50, -57, -59, -60, -61, -63, -64, ...    9000   
A01097     N  [-83, -83, -83, -83, -83, -83, -83, -82, -81, ...    9000   
A03639     N  [-41, -55, 

In [25]:
test_dataset["predictions"] = test_dataset["predictions"].astype(int)

In [29]:
i = 100

print(test_dataset.iloc[i]["class"])
nd.plot_ecg(test_dataset.iloc[i]["data"], fs=300)

O


C:\Users\daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.0.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


<Figure size 1169x827 with 3 Axes>

In [28]:
conf_mat = confusion_matrix(test_dataset["class_index"], test_dataset["predictions"])

def F1_ind(conf_mat, ind):
    return (2 * conf_mat[ind, ind])/(np.sum(conf_mat[ind]) + np.sum(conf_mat[:, ind]))

print(f"Normal F1: {F1_ind(conf_mat, 0)}")
print(f"Other F1: {F1_ind(conf_mat, 1)}")

ConfusionMatrixDisplay.from_predictions(test_dataset["class_index"], test_dataset["predictions"], display_labels=["sufficint quality", "insufficient quality"], cmap="inferno")
plt.show()

Normal F1: 0.5357737104825291
Other F1: 0.057432432432432436


## Try Training the SVM on the physionet data ourselves and see what the results are

In [9]:
ecg_features = []
i = 0

for ind, x in dataset.iterrows():
    print(f"Processing ecg {i}/{len(dataset.index)}\r", end="")
    df = nd._process_ecg(x["data"], fs=300, filter=False)
    df["class_index"] = x["class_index"]
    ecg_features.append(df)
    i += 1


df = pd.concat(ecg_features, keys=ecg_features[0].index)
new_ind = [(dataset.index[i1], i2) for i1, i2 in df.index] # idk how to apply this
df.index = pd.MultiIndex.from_tuples(new_ind)

print(df[df["class_index"] == 1])


               sSQI      kSQI      pSQI    basSQI      bSQI      rSQI  \
A00022 0   0.386698 -0.201232  0.537714  0.746258  0.454545  0.687500   
       1   0.478203 -0.104672  0.538129  0.734294  0.454545  0.647059   
       2   0.666844  0.163131  0.520114  0.724879  0.416667  0.666667   
       3   0.863999  0.392863  0.534983  0.774404  0.500000  0.631579   
       4   1.159561  0.733671  0.549038  0.804093  0.454545  0.647059   
...             ...       ...       ...       ...       ...       ...   
A00056 47 -0.981852  0.093458  0.453477  0.721135  0.636364  0.611111   
       48 -1.049735  0.136307  0.455491  0.785622  0.500000  0.600000   
       49 -0.826705 -0.234170  0.469227  0.758174  0.461538  0.684211   
       50 -0.448652 -0.430022  0.443243  0.666815  0.384615  0.684211   
       51 -0.357077 -0.519726  0.463994  0.711724  0.083333  0.923077   

           class_index  
A00022 0             1  
       1             1  
       2             1  
       3             1 

In [10]:
pk_path = "CinC2017Data/database_yeov_processed.pk"
df.to_pickle(pk_path)

In [11]:
df = pd.read_pickle("CinC2017Data/database_yeov_processed.pk")

In [12]:
print(df)

               sSQI      kSQI      pSQI    basSQI      bSQI      rSQI  \
A00001 0  -0.035735 -0.293814  0.473548  0.860109  0.666667  0.600000   
       1   0.025660 -0.183028  0.437098  0.851583  0.666667  0.600000   
       2   0.123664 -0.188683  0.420040  0.847567  0.666667  0.600000   
       3   0.153531 -0.032273  0.445362  0.925915  0.750000  0.571429   
       4   1.354541  0.397230  0.424141  0.971348  0.750000  0.571429   
...             ...       ...       ...       ...       ...       ...   
A00073 47  0.664680 -0.242644  0.403188  0.956641  0.500000  0.666667   
       48  0.617338 -0.270466  0.396478  0.982734  0.428571  0.700000   
       49  0.615728 -0.272595  0.439226  1.015275  0.466667  0.681818   
       50  0.433565 -0.339626  0.442333  0.954123  0.500000  0.666667   
       51  0.318182 -0.394299  0.468711  0.976110  0.466667  0.681818   

           class_index  
A00001 0             0  
       1             0  
       2             0  
       3             0 

In [13]:
# Normalise some features to have 0 mean and variance 1 (other elements are already limited/normalised)

var_1_features = ["sSQI", "kSQI"]
for f in var_1_features:
    df[f] = (df[f] - df[f].mean())/df[f].var()

In [14]:
train_dataset, test_dataset = train_test_split(df, test_size=0.15, stratify=df["class_index"])


In [29]:
# train the model

class_weights = 1/train_dataset["class_index"].value_counts()
class_weights /= np.sqrt(np.dot(class_weights, class_weights))
print(class_weights.to_dict())

model = SVC(class_weight=class_weights.to_dict())
# See documentation for default values e.g. use rbf, regularising C = 1
features = ["sSQI", "kSQI", "pSQI", "basSQI", "bSQI", "rSQI"]

model = model.fit(train_dataset[features].values, train_dataset["class_index"])

{0: 0.06131627964429509, 1: 0.9981183866909689}


In [39]:
print(test_dataset[features].head().values)

[[-1.77872022  1.32152423  0.33303624  0.99841087  0.75        0.57142857]
 [ 0.37924746 -0.3553033   0.7110753   0.78653168  0.45454545  0.6875    ]
 [-0.03966704 -1.78687679  0.32463987  0.94377819  0.46666667  0.68181818]
 [ 0.63063544  0.1539459   0.58617299  0.90635761  0.5         0.66666667]
 [-0.78811383 -1.22603708  0.25850137  0.9184541   0.45454545  0.6875    ]]


In [ ]:
predictions = model.predict(test_dataset[features].values)

ConfusionMatrixDisplay.from_predictions(test_dataset["class_index"], predictions, display_labels=["sufficint quality", "insufficient quality"], cmap="inferno")
plt.show()
conf_mat = confusion_matrix(test_dataset["class_index"], predictions)

def F1_ind(conf_mat, ind):
    return (2 * conf_mat[ind, ind])/(np.sum(conf_mat[ind]) + np.sum(conf_mat[:, ind]))

print(f"Normal F1: {F1_ind(conf_mat, 0)}")
print(f"Other F1: {F1_ind(conf_mat, 1)}")
